# Setup

Use pip to download and install the necessary libraries if needed

In [ ]:
!pip install --upgrade pma_python
!pip install --upgrade pprint

Import libraries and set connection parameters

In [21]:
# helper libraries
import pprint as pp    # pretty print library is better to print list and dictionary structures
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
import pandas as pd

# pma_python
from pma_python2 import core
print("pma_python library loaded; version", core.__version__)

# connection parameters to be used throughout this notebook
pma_core_server = "http://host.pathomation.com/sandbox/2/PMA.core"
pma_core_user = "user1"
pma_core_pass = "Pathomation"
pma_core_slide_dir = "hgx_cases"

if not core.is_lite(pma_core_server):
    print ("PMA.core found. Good")
else:
    raise Exception("Unable to detect PMA.core! Please update configuration parameters in this block")

pma_python library loaded; version 2.0.0.84
PMA.core found. Good


In [8]:
# only needed when debugging code for extra error messages:
core.set_debug_flag(True)

Debug flag enabled. You will receive extra feedback and messages from pma_python (like this one)


# PMA.core examples

example 10: identifying PMA.core

In [3]:
# testing actual "full" PMA.core instance that may or may not be out there
print("Are you running PMA.core at ", pma_core_server , "?", not core.is_lite(pma_core_server))
# testing PMA.core at non-sense location
print("Are you running PMA.core at http://nowhere ?", core.is_lite("http://nowhere"));

Are you running PMA.core at  http://host.pathomation.com/sandbox/2/PMA.core ? True
Are you running PMA.core at http://nowhere ? None


example 20: getting version information about PMA.core

In [4]:
# assuming we have PMA.core running; what's the version number?
print("Investigating", pma_core_server)
print("You are running PMA.core version", core.get_version_info(pma_core_server))

version = core.get_version_info("http://nowhere");
print (version)
if (version == None):
	print("Unable to detect PMA.core at specified location (http://nowhere/)")
else:
	print("You are running PMA.core version", version);

Investigating http://host.pathomation.com/sandbox/2/PMA.core
http://host.pathomation.com/sandbox/2/PMA.core/api/json/GetVersionInfo
You are running PMA.core version 2.0.0.1386
http://nowhere/api/json/GetVersionInfo
None
Unable to detect PMA.core at specified location (http://nowhere/)


example 30: connect to PMA.core

In [9]:
sessionID = core.connect(pma_core_server, pma_core_user, pma_core_pass)	

if (sessionID == None):
    # PHP has this better: 	echo "User [$pma_core_user] was unable to connect to PMA.core at specified location ($pma_core_server); password used = [$pma_core_pass]".PHP_EOL;
	print("Unable to connect to PMA.core");
else:
	print("Successfully connected to PMA.core; sessionID", sessionID)

Successfully connected to PMA.core; sessionID nBDak_rdY1iKKbfE4y5i9Q2


example 40: getting root-directories from PMA.core

In [10]:
print("You have the following root-directories on your system: ")
rootdirs = core.get_root_directories(sessionID)
pp.pprint(rootdirs)

You have the following root-directories on your system: 
['hgx_cases']


example 50: disconnect from PMA.core

In [23]:
sessionID = core.connect(pma_core_server, pma_core_user, pma_core_pass)	
pp.pprint(core.who_am_i(sessionID))
print()

core.disconnect(sessionID)
try:
    pp.pprint(core.who_am_i(sessionID))
    print()
except:
    print("Unable to identify you. Are you connected to the PMA.core instance? Is that a valid sessionID?")
print()
    
sessionID = core.connect(pma_core_server, pma_core_user, pma_core_pass)	
pp.pprint(core.who_am_i(sessionID))

{'amountOfDataDownloaded': 8,
 'sessionID': '6NQ80b5X5ZyJ94kSqXeAsw2',
 'url': 'http://host.pathomation.com/sandbox/2/PMA.core/',
 'username': 'user1'}

Unable to identify you. Are you connected to the PMA.core instance? Is that a valid sessionID?

{'amountOfDataDownloaded': 8,
 'sessionID': 'aGZSSzuUGn4m0PoGCOdVhw2',
 'url': 'http://host.pathomation.com/sandbox/2/PMA.core/',
 'username': 'user1'}


example 60: getting directories from PMA.core

In [ ]:
rootdirs = core.get_root_directories(sessionID);
print("Directories found in ", rootdirs[0],":")

dirs = core.get_directories(rootdirs[0], sessionID)
pp.pprint(dirs)

example 70: get first non empty directory PMA.core

In [ ]:
slide_dir = core.get_first_non_empty_directory()
print (slide_dir)

example 80: getting slides PMA.core

In [ ]:
slide_dir = pma_core_slide_dir
print("Looking for slides in " + slide_dir)
print()

print ("**Non-recursive:")
print(core.get_slides(slide_dir))

print ("\n**One-level deep recursion:")
print(core.get_slides(slide_dir, recursive = 1))

print ("\n**Full recursion:")
print(core.get_slides(slide_dir, recursive = True))

example 90: get UID for a slide in PMA.core

In [ ]:
slide_dir = pma_core_slide_dir

print("Looking for slides in", slide_dir)
print()

for slide in core.get_slides(slide_dir):
	print (slide," - ", core.get_uid(slide))

example 100: get fingerprint in PMA.core

In [ ]:
slide_dir = pma_core_slide_dir

print("Looking for slides in " + slide_dir)
print()

for slide in core.get_slides(slide_dir):
	print (slide," - ", core.get_fingerprint(slide))

example 110: SlideInfo PMA.core

In [ ]:
slide_dir = pma_core_slide_dir
print("Looking for slides in", slide_dir)
print()

for slide in core.get_slides(slide_dir):
    print("***", slide)
    try:
        pp.pprint(core.get_slide_info(slide))
    except:
        print("**Unable to get slide info from this one")

example 120: slide dimensions PMA.core

In [18]:
slide_dir = pma_core_slide_dir

for slide in core.get_slides(slide_dir):
    print("[" + slide + "]")
    try:
        xdim_pix, ydim_pix = core.get_pixel_dimensions(slide)
        xdim_phys, ydim_phys = core.get_physical_dimensions(slide)

        print("Pixel dimensions of slide: ", end="")
        print(xdim_pix, "x", ydim_pix)

        print("Slide surface area represented by image: ", end="")
        print(str(xdim_phys) + "µm x " + str(ydim_phys) + "µm = ", end="")
        print(xdim_phys * ydim_phys / 1E6, " mm2")
        
    except:
        print("**Unable to parse", slide)

[hgx_cases/CMU-1 - Copy.svs]
Pixel dimensions of slide: 46000 x 32914
Slide surface area represented by image: 22954.0µm x 16424.086µm = 376.998470044  mm2
[hgx_cases/CMU-1.mrxs]
Pixel dimensions of slide: 68608 x 95232
Slide surface area represented by image: 15951.36µm x 22141.440000000002µm = 353.18608035840003  mm2
[hgx_cases/CMU-1.ndpi]
Pixel dimensions of slide: 51200 x 38144
Slide surface area represented by image: 23368.324965769054µm x 17357.906712172924µm = 405.6252047755608  mm2
[hgx_cases/CMU-1.svs]
Pixel dimensions of slide: 46000 x 32914
Slide surface area represented by image: 22954.0µm x 16424.086µm = 376.998470044  mm2
[hgx_cases/CMU-1.tiff]
Pixel dimensions of slide: 46000 x 32914
Slide surface area represented by image: 46000000.0µm x 32914000.0µm = 1514044000.0  mm2


example 130: get all files that make up a particular slide

In [ ]:
for slide in core.get_slides(pma_core_slide_dir):
    print(slide);
    pp.pprint(core.get_files_for_slide(slide))

example 140: who are you in PMA.core

In [ ]:
core.who_am_i()

example 150: investigate zoomlevels PMA.core

In [ ]:
slide_dir = pma_core_slide_dir

for slide in core.get_slides(slide_dir):
    print("***", slide)
    print("  max zoomlevel:", core.get_max_zoomlevel(slide))
    print("  zoomlevel list:")
    pp.pprint(core.get_zoomlevels_list(slide))
    print("  zoomlevel dictionary:")
    pp.pprint(core.get_zoomlevels_dict(slide))

example 160: investigate magnification and other characteristics PMA.core

In [22]:
slide_dir = pma_core_slide_dir

slide_infos = []     # create blank list (to be converted into a pandas DataFrame later)

for slide in core.get_slides(slide_dir):
	dict = {
		"slide": core.get_slide_file_name(slide),
		"approx_mag": core.get_magnification(slide, exact=False),
		"exact_mag": core.get_magnification(slide, exact=True),
		"is_fluo": core.is_fluorescent(slide),
		"is_zstack": core.is_z_stack(slide)
		}
	slide_infos.append(dict)
	
df_slides = pd.DataFrame(slide_infos, columns=["slide","approx_mag","exact_mag", "is_fluo", "is_zstack"])
print(df_slides)


              slide  approx_mag  exact_mag  is_fluo  is_zstack
0  CMU-1 - Copy.svs          20         20    False      False
1        CMU-1.mrxs          40         43    False      False
2        CMU-1.ndpi          20         22    False      False
3         CMU-1.svs          20         20    False      False
4        CMU-1.tiff           0          0    False      False


example 170: get barcode from slide in PMA.core

In [ ]:
for slide in core.get_slides(pma_core_slide_dir, recursive=True):
    print(slide, " - ", core.get_barcode_text(slide))

example 180: show a slide through PMA.core

In [ ]:
slides = core.get_slides(pma_core_slide_dir)
core.show_slide(slides[0])